# Exercise 03: CUQIpy Models, Likelihood and Forward UQ

In this notebook we start to get our hands dirty with modelling inverse problems in CUQIpy. In particular, show how to define new models in CUQIpy from either matricies or functions (methods in python).

## Learning objectives
* Make a CUQIpy model from an existing matrix or function.
* Access pre-defined models from the CUQIpy library.
* Run a simple forward UQ analysis.
* Learn about cuqipy geometries in the context of models.

## Table of contents
1. [Pre-defined models](#pre-defined)
2. [Generating data](#data)
3. [Forward UQ](#forwardUQ)
4. [Creating CUQIpy models](#models)
    1. [Defining model from a matrix](#matrix)
    2. [Defining model from a function](#function)

## 1. Creating CUQIpy models

Before getting started we import the basic packages we need.

In [ ]:
import numpy as np
import cuqi

%load_ext autoreload
%autoreload 2

### 1.A Defining model from a matrix

Suppose we have a linear inverse problem

$$ \mathbf{b}=\mathbf{A}\mathbf{x}+\mathbf{e}, $$

where $\mathbf{b}\in\mathbb{R}^m$ is the measured data, $\mathbf{A}\in\mathbb{R}^{m\times n}$ is a matrix representing the forward model, $\mathbf{x}\in\mathbb{R}^n$ is the unknown (solution) and $\mathbf{e}\in\mathbb{R}^m$ is the additive measurement noise. 

The model is represented by the matrix $\mathbf{A}$ in this case. For the sake of presentation, let us just create a random matrix to represent the forward model.

In [ ]:
#Create a random numpy matrix to act like a forward model (this matrix can be replaced to represent other problems)
n = 10; m = 5
A = np.random.randn(m,n) 

To create a cuqi model represented by this matrix, all we have to do is pass it to the `LinearModel` class from the `model` module in cuqipy as follows.

In [ ]:
model = cuqi.model.LinearModel(A)

This may seem like a superfluous step. However, the cuqipy models have a number of very useful features. Initially let us just have a look at the printed information when we inspect the model. For example we should see that the model have been equipped with domain and range geometries.

In [ ]:
model

#### Try yourself (optional):  
Let A be sudoku matrix....

**Hint:**

In [ ]:
# This is where you type the code:




### 1.B Defining model from a function
We can also define CUQIpy models from functions (methods in python).

...

In [ ]:
def my_func(x):
    return np.sum(x)
model2 = cuqi.model.Model(my_func,range_geometry=m,domain_geometry=n)
model2

Make sodoku out of function instead?? Perhaps some non-linear stuff? Perhaps we move this to end..

## Data generation (Likelihood)
From the problem info string above, we see that the noise is additive Gaussian. Hence, we need to define a Gaussian likelihood, with the model as mean. This is easily done as follows.

Math about mean=model...

show pdf of Gaussian with model(x).

In [ ]:
likelihood = cuqi.distribution.Gaussian(mean=model,std=0.05)

Note in particular that likelihood is a conditional distribution. Conditional on the input parameter to the model (in this case x). This can be seen by checking

In [ ]:
likelihood.get_conditioning_variables()

Demonstrate likelihood with zero image. (noise). Tell them to simulate data..

#### Try yourself (optional):  
Try computing some relalizations of the noise. 

**Hint:** What is $\mathbf{x}$ if we are only interested in the noise?

In [ ]:
# This is where you type the code:




## 2 Forward UQ
Suppose we have generated some samples from a Gaussian Markov Random Field and aim to see effect of pushing this distribution through the linear model from earlier.

First lets define the distribution, generate some samples and plot them

In [ ]:
Ns = 50; #Number of samples (try changing this to improve the confidence interval)
x = cuqi.distribution.GMRF(mean=np.zeros(n),prec=1,partition_size=n,physical_dim=1,bc_type='zero')
xs = x.sample(Ns)
xs.plot_ci(95)

Now we compute the forward projection of each sample and plotting the resulting pushed forward samples.

In [ ]:
bs = model(xs)
bs.plot_ci(95)

#### Try yourself (optional):  
This above confidence interval plot can be a bit misleading as we only have a few output parameters. Try modifing the `range_geometry` of the model into a discrete geometry.

**Hint:** See `help(cuqi.geometry.Discrete)` for how to define a discrete geometry.

In [ ]:
# This is where you type the code:




In [ ]:
# Recomputing the forward projection after the model geometry is updated. This plot below should look different!
bs = model(xs)
bs.plot_ci(95)

## 3. Pre-defined models
In CUQIpy we also have a number of pre-defined models. For example:

In [ ]:
modelD, data, probInfo = cuqi.testproblem.Deconvolution.get_components()
modelD

In [ ]:
#TODO:
#Improve doc on LinearModel and model.forward